## Knapsack Problem

The *knapsack problem*:  given a set of integers $S = \{s_1, s_2, \ldots, s_n\}$ and a target number $T$, find a subset (i.e., knapsack) of $S$ which adds up exactly to $T$.  

For example, if $S = \{1,2,5, 9, 10\}$, there is a subset that adds up to $T = 22$, but not to $T = 23$.  Complete the following tasks related to this problem.

# 1. 

Find a subset of $S = \{1,2,5, 9, 10\}$ with sum $T = 22$.  Explain the process (algorithm) you used mentally to find the subset.  Then apply the same process in an attempt to find a subset with sum $T = 23$.  

How do you know there is no such subset?

# 2.

Consider the following possible algorithm for the knapsack problem, written in psuedocode: 
```python
knapsack(S[], T):
    K = empty
    for each i < size(S)
        if sum(K) + S[i] <= T, put S[i] into K
    if sum(K) = T, return K, else return False.
```
**a)** Describe what this algorithm does in English.  

**b)** Implement this algorithm in Python and run it on the $S$ and $T$ above.


**c)** Prove that this algorithm is NOT correct.  That is, find a counterexample: a set $S$ and number $T$ for which there is a solution, but not one that the algorithm finds.

**d)** Verify that this particular $S$ and $T$ does not give the right output when entered to your Python program.

# 3. 

Another try: What if you put the elements in the knapsack from largest to smallest?  Check that this too is not a correct algorithm.

# 4.

Describe a correct algorithm for the knapsack problem (that we haven't seen in class), both in English and in pseudocode.  Then implement the algorithm in Python.  Explain how you know your algorithm is correct (even if it might not be efficient).

# 5. Generating correct change

Now, we will be making change using the fewest coins. 

Suppose you are a programmer for a vending machine manufacturer. Your company wants to streamline effort by giving out the fewest possible coins in change for each transaction. Suppose a customer puts in a dollar bill and purchases an item for 37 cents. What is the smallest number of coins you can use to make change? The answer is six coins: two quarters, one dime, and three pennies. 

How did we arrive at the answer of six coins? We start with the largest coin in our arsenal (a quarter) and use as many of those as possible, then we go to the next lowest coin value and use as many of those as possible. This is the greedy algorithm for change-making.

**Question:** Write the greedy algorithm for change making.

The input is the amount of change to generate (in pennies) and a list of coin sizes (in pennies)

The output is the minimum number of coins to gener

```
# buys with 1 dollar for 37 pennies
# Second argument says we can give quarters, dimes, nickels and pennies
make_change(100 - 37, [25, 10, 5, 1])

# 2 quarters, one dime, and three pennies
output --> 6 # Output would be equivalent to the choices [2, 1, 0, 3]
```

# 6 Recursive change

Write the greedy change making algorithm using recursion

# 7 Dynamic Programming Change making

Write a solution to the change making problem using dynamic programming.

**Hint:** Start with making change for one cent and systematically work its way up to the amount of change we require. This guarantees us that at each step of the algorithm we already know the minimum number of coins needed to make change for any smaller amount. Keep a memoized table of results for each step working up to the amount of change you need to generate.